## Honest Party execution

In [19]:
import csv
import pandas as pd
import glob

In [20]:
def filter_first_le_value(df, low):
    filtered_df = df[df['CumBits'] > low]
    if not filtered_df.empty:
        return filtered_df.iloc[0]
    else:
        return None

## Monte Carlo Analysis and Probability Calculations

In [222]:
folder_name_honest="/home/gop/quai/POEM-analysis/poem_db/11g800mspoem/poemhonest1/*/34.31.199.132/simulations/"
folder_name_adv="/home/gop/quai/POEM-analysis/poem_db/11g800mspoem/poemadv1/*/34.72.118.123/simulations/"
folder_honest = glob.glob(folder_name_honest, recursive=True)[0]
folder_adv = glob.glob(folder_name_adv, recursive=True)[0]

In [214]:
import math
x_bits=[]
TOTAL_BLOCKS=200
TOTAL_ROUNDS=100
#threshold=math.log(25000)
threshold=11
for i in range(0, TOTAL_BLOCKS):
    x_bits.append((threshold + 1/math.log(2))*i)

In [215]:
import math

def compute_one_round(i):
    # Honest Party
    honest_file=folder_honest+f"{i}"+".csv"
    df1 = pd.read_csv(honest_file, skiprows=1)
    df1['Delta Timestamp']=df1[' Time Stamp'].diff()/1000
    df1['Delta Timestamp']=df1['Delta Timestamp'].fillna(0)
    df1['CumBits']=df1[' Extra Bits'].cumsum()
    df1['CumTime']=df1['Delta Timestamp'].cumsum()

    columns=['Bits', 'Time', 'CumBits']
    df1_final = pd.DataFrame(columns=columns)

    cum_honest_times=[]
    cum_honest_bits=[]
    for bit in x_bits:
        value=filter_first_le_value(df1, bit)
        if value is not None:
            cum_honest_times.append(value.CumTime)
            cum_honest_bits.append(value.CumBits)
        else:
            cum_honest_times.append(10000)
            cum_honest_bits.append(0)
    
    df1_final['Bits']=x_bits
    df1_final['Time']=cum_honest_times
    df1_final['CumBits']=cum_honest_bits

    # Adversary
    adv_file=folder_adv+f"{i}"+".csv"
    df2 = pd.read_csv(adv_file, skiprows=1)
    df2['Delta Timestamp']=df2[' Time Stamp'].diff()/1000
    df2['Delta Timestamp']=df2['Delta Timestamp'].fillna(0)
    df2['CumBits']=df2[' Extra Bits'].cumsum()
    df2['CumTime']=df2['Delta Timestamp'].cumsum()

    columns=['Bits', 'Time', 'CumBits']
    df2_final = pd.DataFrame(columns=columns)

    cum_adv_times=[]
    cum_adv_bits=[]
    for bit in x_bits:
        value=filter_first_le_value(df2, bit)
        if value is not None:
            cum_adv_times.append(value.CumTime)
            cum_adv_bits.append(value.CumBits)
        else:
            cum_adv_times.append(10000)
            cum_adv_bits.append(0)

    df2_final['Bits']=x_bits
    df2_final['Time']=cum_adv_times
    df2_final['CumBits']=cum_adv_bits

    results = []
    
    for bits in df1_final['Bits']:
        row_df1 = df1_final[df1_final['Bits'] == bits]
        row_df2 = df2_final[df2_final['Bits'] == bits]

        winner = 1 if row_df1['Time'].iloc[0] < row_df2['Time'].iloc[0] else 0
        results.append([bits, winner])

    winning_df = pd.DataFrame(results, columns=['Bits', 'Winner'])

    return winning_df

In [216]:
all_dfs=[]
for i in range(1, TOTAL_ROUNDS+1):
    all_dfs.append(compute_one_round(i))

In [217]:
cumbits=0
cumtime=0
for i in range(1, TOTAL_ROUNDS+1):
    honest_file=folder_honest+f"{i}"+".csv"
    df1 = pd.read_csv(honest_file, skiprows=1)
    cumbits+=df1[' Extra Bits'].sum()
    df1['Delta Timestamp']=df1[' Time Stamp'].diff()/1000
    df1['Delta Timestamp']=df1['Delta Timestamp'].fillna(0)
    df1['CumTime']=df1['Delta Timestamp'].cumsum()
    cumtime+=df1['CumTime'].tail(2).iloc[0]
    
average_time_per_sim=cumtime/TOTAL_ROUNDS
f=TOTAL_BLOCKS/average_time_per_sim
average_bits=cumbits/TOTAL_BLOCKS/TOTAL_ROUNDS
print("f", f, "Average Bits", average_bits)

f 0.5995999708834254 Average Bits 11.958843610771334


In [218]:
total_winner=[]
for bits in x_bits:
    result=0
    for df in all_dfs:
        for value in df[df['Bits']==bits]['Winner']:
            result+=value
    total_winner.append(result)

columns=['Bits', 'Total Wins']
m_final = pd.DataFrame(columns=columns)
m_final['Bits']=x_bits
m_final['Total Wins']=total_winner

In [219]:
expected_number_of_blocks=(m_final[m_final['Total Wins'] > 89].iloc[0].Bits)/(threshold + 1/math.log(2))
print("Expected Number of Blocks", expected_number_of_blocks)

Expected Number of Blocks 21.0


In [220]:
d=(expected_number_of_blocks/f)

In [221]:
print("Confirmation Delay", d)

Confirmation Delay 35.0233506
